# NRL Comp setup

## Setting up markets and bots:
- On clean Heroku database
- Create a user for createMarket and for the bots.
- Set up markets on Monday night with name like 'Event StGeorge v NewCastle 2019-05-19T16:00:00-10:00 Betfair 1.158610323'. Time is ISO8601 with a UTC offset to go to Brisbane time. Outcome between 0 and 10000 (10000 = StGeorge win, 0 = NewCastle win )
- Setup bot with BetfairPriceMakingBot(marketId) looks at the market description and sets itself up. These will keep quoting until they encounter an error or run out of money. If they go down, put up again manually.
- Settle all markets at the end of the week.



In [1]:
# Basic bot 
import requests
import pandas as pd
import json
import threading
from BlocBot import BlocBot
from BetfairPriceMakingBot import BetfairPriceMakingBot


In [2]:
# Create a new user
# baseurl = 'http://127.0.0.1:7000/'
baseurl = 'https://blocmarket.herokuapp.com/'
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content))

In [3]:
response.json()

{'signingKey': '9d7ddb72ce50043ef46d0d2490eeeb169311da3773b2e78a512b536c48736b8a',
 'traderId': '9',
 'verifyKey': '93ce2d3fb226bceb59ec59ef98b41db6e91994deffbdde625a72a9c54f1c067c'}

In [4]:
# Save user details
sk = response.json()['signingKey']
vk = response.json()['verifyKey']
tId = int(response.json()['traderId'])

In [5]:
# Create all the markets
url = baseurl + 'createMarket'
headers = {'content-type': 'application/json'}

# Market descriptions 
marketD = ['Event Canberra v Cronulla 2019-06-01T16:00:00-10:00 betfair 1.159512374',
          'Event GoldCoast v NZWarriors 2019-06-01T16:00:00-10:00 betfair 1.159512658']
          
mrtId = 0
allMarketIds = []
for md in marketD:
    mrtId = mrtId+1
    content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": mrtId, 
                        "marketBranchId": 1, 
                        "marketMin": 0,
                        "marketMax": 10000,
                        "marketDesc": md}
    # Post market
    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    allMarketIds.append(response.json()['marketId'])


In [6]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '2', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 2,
 'marketMax': 10000,
 'marketMin': 0,
 'marketRootId': 2,
 'traderId': 9}

In [7]:
# Basic Bot definition
botdef = {'spread': 0.01,
 'updateFrequencySeconds': 30,
 'verifyKey': vk,
 'signingKey':  sk,
 'traderId': tId,
 'multiplier': 10000,
 'blocurl': baseurl,
 'blocheaders': {'content-type': 'application/json'}}

botdef

{'spread': 0.01,
 'updateFrequencySeconds': 30,
 'verifyKey': '93ce2d3fb226bceb59ec59ef98b41db6e91994deffbdde625a72a9c54f1c067c',
 'signingKey': '9d7ddb72ce50043ef46d0d2490eeeb169311da3773b2e78a512b536c48736b8a',
 'traderId': 9,
 'multiplier': 10000,
 'blocurl': 'https://blocmarket.herokuapp.com/',
 'blocheaders': {'content-type': 'application/json'}}

In [8]:
bfPass = 'ees'

# Run single bot

In [9]:
# Create a bot for market (Make sure VPN location is OK)
iMarket = 1
bot = BetfairPriceMakingBot(iMarket)
bot.getBetfairSessionToken(betfairAppKey ='iw8UsiHCP1GSs213', betfairPassword=bfPass)
    
# Add some defs
for key,val in botdef.items():
    # Set bot inputs from definiton
    setattr(bot, key, val)
    


In [ ]:
# Check quot
bot.getQuote()

In [ ]:
# Stream qutoe
bot.streamQuote()

[Within cells interlinked]: quote 5277.375515205525
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Wi

# Run multiple bots on threads


In [10]:
# Set up bot threads
threads = []
for iMarket in range(len(marketD)):
    
    # Create a bot for each market (Make sure VPN location is OK)
    bot = BetfairPriceMakingBot(iMarket+1)
    bot.getBetfairSessionToken(betfairAppKey ='iw8UsiHCP1GSs213', betfairPassword=bfPass)
    
    # Add some defs
    for key,val in botdef.items():
        # Set bot inputs from definiton
        setattr(bot, key, val)
    
    t = threading.Thread(target=bot.streamQuote, args=())
    threads.append(t)


In [11]:
# Start threads
for t in threads:
    t.start()
    

In [12]:
threads

[<Thread(Thread-4, started 123145335468032)>,
 <Thread(Thread-5, started 123145340723200)>]

In [13]:
bot

In [14]:
vars(bot)

{'multiplier': 10000,
 'spread': 0.01,
 'updateFrequencySeconds': 30,
 'verifyKey': '072be7fe4e3ae8758e49dc165d98ce1855a90d50c7f7afdc70a99f4b0a34737c',
 'signingKey': 'fe7e21df77c46cb2d6aaa810145c535cca9a1c82fa90ff7ea8195b7b89c1a30d',
 'traderId': 1,
 'marketId': 2,
 'blocurl': 'https://blocmarket.herokuapp.com/',
 'blocheaders': {'content-type': 'application/json'},
 'quoteSource': 'betfair',
 'betfairMarketId': '1.159512658',
 'betfairAppKey': 'iw8UsiHCP1GSs213',
 'betfairSessionToken': 'wlol+/vxBUuoVzTFBWprrGajmj6GEOTCF5YRqcEDUZQ=',
 'blocUrl': 'https://blocmarket.herokuapp.com/'}

[Within cells interlinked]: 5353.104514356053
[Within cells interlinked]: 4861.701123383366


In [16]:
pd.read_json(bot.getTradeSummary())

,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC,marketRootId,marketBranchId,marketMin,marketMax,marketMinOutcome,marketMaxOutcome
0,2,1,5299,1,1,True,2019-06-11 19:46:06.898,1,1,0,10000,-5299,4701
1,5,1,5299,-1,1,True,2019-06-11 19:46:42.049,1,1,0,10000,5299,-4701
2,1,2,4813,1,1,True,2019-06-11 19:46:06.892,2,1,0,10000,-4813,5187
3,6,2,4813,-1,1,True,2019-06-11 19:46:43.235,2,1,0,10000,4813,-5187
4,3,1,5406,-1,1,True,2019-06-11 19:46:08.422,1,1,0,10000,5406,-4594
5,7,1,5406,1,1,True,2019-06-11 19:46:43.671,1,1,0,10000,-5406,4594
6,4,2,4910,-1,1,True,2019-06-11 19:46:09.588,2,1,0,10000,4910,-5090
7,8,2,4910,1,1,True,2019-06-11 19:46:44.800,2,1,0,10000,-4910,5090


In [13]:
bot.getQuote()

{'Bid': 5181.347150259068,
 'Ask': 5263.157894736842,
 'Trade': 5222.2525224979545,
 'TimeStampUTC': datetime.datetime(2019, 6, 12, 13, 44, 52, 856033)}

In [14]:
vars(bot)

{'multiplier': 10000,
 'spread': 0.01,
 'updateFrequencySeconds': 30,
 'verifyKey': 'd60fde610a83bcbf15e18150bc2adaac79468c799b56b7d87d4668efa4927057',
 'signingKey': 'c4b9751b499fb68fceeb05e2b7e404b9644f7146249d4d74c413d0284e15a64c',
 'traderId': 1,
 'marketId': 1,
 'blocurl': 'https://blocmarket.herokuapp.com/',
 'blocheaders': {'content-type': 'application/json'},
 'quoteSource': 'betfair',
 'betfairMarketId': '1.159512374',
 'betfairAppKey': 'iw8UsiHCP1GSs213',
 'betfairSessionToken': 'gofoL0udQa095Gw0fB84iHQ6Z1VYtrHqZKUz88Tkz1o=',
 'blocUrl': 'https://blocmarket.herokuapp.com/'}